In [10]:
# Tell the notebook where to find the pipeline functions
import sys
sys.path.append('/lab-share/Neuro-Cohen-e2/Public/notebooks/gmiller/Pipeline/0_pipeline_scripts')

In [1]:
import os
import shutil
from glob import glob 
import GNMpipeline as p

# Add your inputs

In [2]:
input_dir="RAW/"  #Folder with input files
input_type='Folder' #'BIDS' or 'Folder'  
output_dir="output_jpeters_20240620_test"

subjects_to_skip=['c002', 'c003'] #If you don't have any to skip put None


IMAGE_TYPES = ['t1', 'T2', 'FLAIR'] #case sensitive, change to match what you used e.g. t1, t1w, TI 

# Make dictionary of subjects and sessions

In [3]:
subject_sessions=p.create_input_dict(input_dir, subjects_to_skip=subjects_to_skip, input_type=input_type)

There are 1 unique subjects to be registered
{'c001': ['s01']}


# Visualize output folder system

In [4]:
print(f'Your output folder, {output_dir}, will look something like this: ')
p.print_tree(subject_sessions)

Your output folder, output_jpeters_20240620_test, will look something like this: 
c001
    s01


# Create output folder system

In [5]:
# Create output folders for each subject and session based on the subject_sessions dictionary
for subject, sessions in subject_sessions.items():
    
    subject_folder = os.path.join(output_dir, subject)
    
    # Create the subject folder if it doesn't exist
    if not os.path.exists(subject_folder):
        os.makedirs(subject_folder)
    
    
    # Loop through each session for the subject
    for session in sorted(set(sessions)):
        print(f'Preparing output folders for {session} for {subject}')
        session_folder = os.path.join(subject_folder, session)
         
        # Check if the session folder exists and is not empty
        if os.path.exists(session_folder):
            if os.listdir(session_folder):
                print(f"{subject}: {session_folder} exists and is not empty")
        else:
            # Create the session folder if it doesn't exist
            if not os.path.exists(session_folder):
                os.makedirs(session_folder)

Preparing output folders for s01 for c001
c001: output_jpeters_20240620_test/c001/s01 exists and is not empty


# Copy files to output directory, combining files as needed

In [6]:
# Copy images to output folders; Combine images as needed
combine_dict={}
for subject, sessions in subject_sessions.items():
    subject_folder = os.path.join(output_dir, subject)
    

    # Loop through each session for the subject
    for session in sorted(set(sessions)):
        session_folder = os.path.join(subject_folder, session)
        
        if not os.path.exists(f'{session_folder}/SELECTED'):
            os.makedirs(f'{session_folder}/SELECTED')
         
        # Loop through each image type 
        for image_type in IMAGE_TYPES:

            # Check if a file with the image type already exists in the session folder
            if glob(f'{session_folder}/SELECTED/*{image_type}*.nii*'):
                    print(f"Error: A file with image type '{image_type}' already exists in {session_folder}/SELECTED")
            else: 
                # Get the list of images for the image type
                if input_type == 'Folder':
                    images = glob(f'{input_dir}/{subject}*{session}*{image_type}*.nii*') 
                elif input_type == 'BIDS':
                    images = glob(f'{input_dir}/{subject}/{session}/*{image_type}*.nii*')
             
                # Check if there are more than 3 images to combine
                if len(images) > 3:
                    print(f"Error: More than 3 images found for participant {subject}, session {session}, and image type {image_type}.")
                    continue
                
                # Combine the images if there are more than 1
                if len(images) > 1 and image_type not in ['FLAIR', 'flair', 'Flair']:
                    
                    #Copy the un-combined images to the output folder so you have them
                    os.makedirs(f'{session_folder}/SELECTED/precombine', exist_ok=True)
                    [shutil.copy(i,f'{session_folder}/SELECTED/precombine') for i in images]
                    
                    #create the combine command and submit it to SLURM
                    print(f"combining: {images}")
                    images_to_combine=glob(f'{session_folder}/SELECTED/precombine/*{image_type}*nii*') 
                    command = p.combine_images(f'{session_folder}/SELECTED', images_to_combine, f'{subject}_{session}_{image_type}', clean_up=True)
                    job_name = f"combine_images_{subject}_{session}_{image_type}"
                    job_id=p.submit_slurm_job(job_name, command)
                    combine_dict[(subject,session)] = job_id
                    
                if len(images) > 1 and image_type in ['FLAIR', 'flair', 'Flair']:
                    print('You have more than one FLAIR image, Combining these may not work well, please select one')
                
                # Copy the single image to the session folder
                elif len(images) == 1:
                    print(f"copying: {images} to {session_folder}/SELECTED/{subject}_{session}_{image_type}.nii.gz")
                    shutil.copy(images[0], f'{session_folder}/SELECTED/{subject}_{session}_{image_type}.nii.gz')

# Print the number of combine jobs submitted to SLURM
if len(combine_dict) > 0:
    print('You have', len(combine_dict), 'Combine Jobs submitted to SLURM; subject and job IDs are stored in combine_dict')
    print('You can type "squeue -u $USER" into your terminal to track SLURM job progress')
    print('You can check the output file matching the jobid in combine_dict to see code outputs and any errors')
else:
    print('You have', len(combine_dict), 'Combine Jobs')

Error: A file with image type 't1' already exists in output_jpeters_20240620_test/c001/s01/SELECTED
Error: A file with image type 'T2' already exists in output_jpeters_20240620_test/c001/s01/SELECTED
Error: A file with image type 'FLAIR' already exists in output_jpeters_20240620_test/c001/s01/SELECTED
You have 0 Combine Jobs


# Check your submitted jobs

In [7]:
print("These are the jobs that should have been submitted:")
print(combine_dict)

These are the jobs that should have been submitted:
{}


In [8]:
!squeue -u $USER 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1587644 bch-inter     bash ch236393  R    5:29:17      1 compute-5-0-3
